In [16]:
import numpy as nm
import pandas as pd
import operator
from collections import Counter, defaultdict
import math
import numpy as np

import nltk

## Loading and Pre-Processing Data

In [3]:
test_tagged_txt = open("data/brown.test.tagged.txt", "r").read()
test_raw = open("data/brown.test.txt", "r").read()

train_raw = open("data/brown.train.tagged.raw", "r").read()
train_tagged_txt = open("data/brown.train.tagged.txt", "r").read()

# Question 2a

In [25]:
train_full = train_tagged_txt.replace('\n', ' ').lower().split()
test_full = test_tagged_txt.replace('\n', ' ').lower().split()
train = {}
train_all = {}
test = []
test_correct_tags = []

for tp in test_full:
    tp = tp.split('/')
    test.append(tp[0])
    test_correct_tags.append(tp[1])

for tp in train_full:
    tp = tp.split('/') #tp[0] = word, tp[1] = tag
    if tp[0] not in train:
        train[tp[0]] = {}
    if tp[1] not in train[tp[0]]:
        train[tp[0]][tp[1]] = 0
    train[tp[0]][tp[1]] += 1
    
for key, value in train.items():
    sorted_dict = sorted(train[key].items(), key=operator.itemgetter(1), reverse=True)
    if key not in train_all:
        train_all[key] = []
    train_all[key] = sorted_dict
    train[key] = sorted_dict[0]

In [4]:
def get_token_tags(txt):
    token_tag_list = txt.lower().replace('\n', ' ').split()
    tokens = []
    tags = []
    for token_tag in token_tag_list:
        pair = token_tag.split('/')
        tokens.append(pair[0])
        tags.append(pair[1])
    return tokens, tags

In [5]:
def count_token_with_tags(tokens, tags):
    token_tags = defaultdict(Counter)
    for token, tag in zip(tokens, tags):
        token_tags[token][tag] += 1
    # normalize
    for token in token_tags:
        count = 0
        for tag in token_tags[token]:
            count += token_tags[token][tag]
        for tag in token_tags[token]:
            token_tags[token][tag] = token_tags[token][tag] / count

    return token_tags 

In [6]:
def normalized_tag_count(tags):
    total = len(tags)
    tag_count = Counter(tags)
    for key in tag_count.keys():
        tag_count[key] = float(tag_count[key]) / total
    return tag_count

In [7]:
def read_text_lines(text):
    sentences = [[word.split('/')[0] for word in sentence.split()] for sentence in text.lower().split('\n')]
    return sentences

In [8]:
def token_array(token_count):
    unique_tokens = []
    for token in token_count:
        unique_tokens.append(token)
    return unique_tokens

## Training set

In [9]:
train_tokens, train_tags = get_token_tags(train_tagged_txt)
train_total = len(train_tokens)
train_wordcount = Counter(train_tokens)
train_tag_token_count = count_token_with_tags(train_tags, train_tokens)
train_tag_count_norm = normalized_tag_count(train_tags)
train_unique_tokens = token_array(train_wordcount)
train_unique_tags = token_array(train_tag_count_norm)

## Test set

In [10]:
test_tokens, test_correct_tags = get_token_tags(test_tagged_txt)
test_sentences = read_text_lines(test_tagged_txt)

## Running on the Test Set

In [11]:
train_token_tag_count = count_token_with_tags(train_tokens, train_tags)
train_token_tag_most = {}
for token in train_token_tag_count:
    max_count = 0
    max_tag = None
    for tag in train_token_tag_count[token]:
        if train_token_tag_count[token][tag] > max_count:
            max_count = train_token_tag_count[token][tag]
            max_tag = tag
    
    train_token_tag_most[token] = max_tag 

new_labels = []

for sentence in test_sentences:
    for token in sentence:
        tag = 'nn'
        if token in train_token_tag_count:
            tag = train_token_tag_most[token]
        new_labels.append(tag)

#for token in test_tokens:
#    tag = 'nn'
#    if token in train_token_tag_count:
#        tag = train_token_tag_most[token]
#    new_labels.append(tag)

In [12]:
df = pd.DataFrame({'test_word': test_tokens, 'new_label': new_labels, 'ground_truth': test_correct_tags})

In [13]:
df

,test_word,new_label,ground_truth
0,in,in,in
1,great,jj,jj
2,expectations,nns,nns
3,the,at,at
4,hands,nns,nns
...,...,...,...
148962,thoughts,nns,nns
148963,off,rp,in
148964,tom,np,np
148965,brannon,np,np


## Accuracy

In [14]:
def accuracy(predicted_labels, correct_labels):    
    correct = 0
    for n, o in zip(predicted_labels, correct_labels):
        if n == o:
            correct += 1

    accuracy = (correct / len(predicted_labels)) * 100.0
    return accuracy

simple_accuracy = accuracy(new_labels, test_correct_tags)
print("Accuracy: %s" % simple_accuracy)

Accuracy: 91.52094087952365


# Question 2b

In [15]:
conf_mtr = {}

for label in test_correct_tags:
    conf_mtr[label] = {}
    for i in test_correct_tags:
        conf_mtr[label][i] = 0

In [17]:
# что-то тут вроде неправильно
for n, o in zip(new_labels, test_correct_tags):
    conf_mtr[o][n] += 1
conf_df = pd.DataFrame.from_dict(conf_mtr, orient='index', columns = test_correct_tags).transpose()

In [18]:
conf_df[:10]

,in,jj,nns,at,vb,rb,nn,.,,cd,...,output,coating,vbg+to,ft,4,32'',16'',vbn+to,or,dt$
in,13829,13,0,0,0,184,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jj,6,7999,0,0,58,152,264,0,0,0,...,0,0,0,0,0,0,0,0,0,0
nns,0,0,6969,0,1,0,31,0,0,1,...,0,0,0,0,0,0,0,0,0,0
at,0,0,0,12747,0,33,12,0,0,0,...,0,0,0,0,0,0,0,0,0,0
nns,0,0,6969,0,1,0,31,0,0,1,...,0,0,0,0,0,0,0,0,0,0
vb,1,21,0,2,3407,0,340,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rb,20,69,0,0,19,3767,36,0,0,0,...,0,0,0,0,0,0,0,0,0,0
at,0,0,0,12747,0,33,12,0,0,0,...,0,0,0,0,0,0,0,0,0,0
nn,21,717,451,1,769,106,20396,0,0,99,...,1,1,0,1,0,0,0,0,0,0
.,0,0,0,0,0,0,0,7833,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
new_labels

['in',
 'jj',
 'nns',
 'at',
 'nns',
 'vb',
 'rb',
 'at',
 'nn',
 '.',
 '',
 'jj',
 'cd',
 '.',
 'in',
 'at',
 'jj',
 'nn',
 ',',
 'jj',
 'vbz',
 'cd',
 'nn',
 ',',
 'nn',
 'pp$',
 'nns',
 'cc',
 'rb',
 'vbz',
 'jj',
 'nn',
 'to',
 'cs',
 'nn',
 '.',
 'qlp',
 'vbg',
 'nns',
 'bed',
 'vbn',
 'to',
 'vb',
 'at',
 'nn',
 'nn',
 'nn',
 '.',
 'vb',
 'at',
 'jj',
 'nn',
 ',',
 'vb',
 'ppo',
 'abn',
 ',',
 'in',
 'nn',
 '.',
 'rb',
 ',',
 'in',
 'nn',
 'jj',
 'in',
 'pp$',
 'nns',
 ',',
 'ppss',
 'vbd',
 'to',
 'at',
 'nn',
 '.',
 'in',
 'np$',
 'nn',
 '.',
 '.',
 'to',
 'hv',
 'np',
 'nn',
 'in',
 'nn',
 'in',
 'pp$',
 'nns',
 '.',
 'vbg',
 'ex',
 'pps',
 'vbd',
 'nns',
 'in',
 'nn',
 'vb',
 'in',
 'at',
 'nns',
 'cc',
 'vb',
 'in',
 'at',
 'nn',
 ',',
 'jj',
 ',',
 'rb',
 'nn',
 ',',
 'pp$',
 'nn',
 'jj',
 ',',
 'rb',
 'vbg',
 'in',
 'vbg',
 'cc',
 'vbg',
 'cs',
 'at',
 'nn',
 'in',
 'np',
 'vbd',
 ',',
 'rb',
 'vbg',
 'at',
 'nn',
 'cs',
 'vbd',
 'in',
 'pp$',
 '.',
 'in',
 'pp$',
 'nn',
 

In [20]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(conf_df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
for t, n, o in zip(test, new_labels, test_correct_tags):
    if n != o:
        print(t + ": correct=" + o + ", new=" + n)

rival: correct=nn, new=jj
buries: correct=vbz, new=nn
to: correct=in, new=to
that: correct=dt, new=cs
enough: correct=ap, new=qlp
12-passenger: correct=jj, new=nn
to: correct=in, new=to
dwelling: correct=vbg, new=nn
there: correct=rb, new=ex
imperious: correct=jj, new=nn
that: correct=wps, new=cs
her: correct=ppo, new=pp$
no: correct=ql, new=at
use: correct=vb, new=nn
so: correct=cs, new=ql
name: correct=vb, new=nn
play: correct=nn, new=vb
little: correct=ql, new=ap
raised: correct=vbd, new=vbn
high: correct=rb, new=jj
asch: correct=np, new=nn
transcultural: correct=jj, new=nn
sturbridge: correct=np, new=nn
completely-restored: correct=jj, new=nn
to: correct=in, new=to
planned: correct=vbd, new=vbn
executive: correct=jj, new=nn
beginning: correct=nn, new=vbg
to: correct=in, new=to
:: correct=in, new=:
:: correct=in, new=:
2048: correct=cd, new=nn
recommended: correct=vbd, new=vbn
act: correct=vb, new=nn
to: correct=in, new=to
out: correct=in, new=rp
sink: correct=vb, new=nn
before: cor

sigemund: correct=np, new=nn
871: correct=cd, new=nn
892: correct=cd, new=nn
cheating: correct=vbg, new=nn
racing: correct=nn, new=vbg
just: correct=jj, new=rb
thought: correct=vbn, new=vbd
through: correct=rp, new=in
caught: correct=vbn, new=vbd
it: correct=ppo, new=pps
faced: correct=vbd, new=vbn
got: correct=vbn, new=vbd
little: correct=ql, new=ap
that: correct=dt, new=cs
loading: correct=nn, new=vbg
lb.: correct=nn, new=nns
you: correct=ppo, new=ppss
potential: correct=nn, new=jj
somewhat: correct=rb, new=ql
to: correct=in, new=to
found: correct=vbd, new=vbn
appeared: correct=vbn, new=vbd
mousy: correct=jj, new=nn
cleaner: correct=jjr, new=nn
means: correct=vbz, new=nns
to: correct=in, new=to
unwitting: correct=jj, new=nn
loose: correct=rb, new=jj
free: correct=vb, new=jj
followed: correct=vbn, new=vbd
to: correct=in, new=to
to: correct=in, new=to
fascinated: correct=vbd, new=vbn
only: correct=ap, new=rb
tangent: correct=nn, new=jj
for: correct=cs, new=in
such: correct=abl, new=jj


as: correct=ql, new=cs
that: correct=wps, new=cs
got: correct=vbn, new=vbd
her: correct=ppo, new=pp$
out: correct=in, new=rp
there: correct=rb, new=ex
one: correct=pn, new=cd
compound: correct=vb, new=nn
it: correct=ppo, new=pps
sours: correct=vbz, new=nn
budget: correct=vb, new=nn
lists: correct=vbz, new=nns
$1.26: correct=nns, new=nn
$1.10: correct=nns, new=nn
$.105: correct=nns, new=nn
opposite: correct=in, new=jj
to: correct=in, new=to
jabberings: correct=nns, new=nn
it: correct=ppo, new=pps
softened: correct=vbd, new=vbn
all: correct=ql, new=abn
alokut: correct=np, new=nn
racing: correct=nn, new=vbg
null-type: correct=jj, new=nn
stretched: correct=vbn, new=vbd
0.001: correct=cd, new=nn
1806: correct=cd, new=nn
realized: correct=vbn, new=vbd
until: correct=in, new=cs
sir: correct=nn, new=np
to: correct=in, new=to
heard: correct=vbn, new=vbd
insurgent: correct=jj, new=nn
model: correct=jj, new=nn
so-far: correct=rb, new=nn
block: correct=vb, new=nn
token: correct=jj, new=nn
rush: co

to: correct=in, new=to
it: correct=ppo, new=pps
charge: correct=vb, new=nn
semi-nude: correct=jj, new=nn
pornographic: correct=jj, new=nn
to: correct=in, new=to
emphysematous: correct=jj, new=nn
blebs: correct=nns, new=nn
to: correct=in, new=to
cm.: correct=nns, new=nn
blows: correct=vbz, new=nns
hearest: correct=vbz, new=nn
dost: correct=do, new=nn
screen: correct=vb, new=nn
represented: correct=vbd, new=vbn
total: correct=jj, new=nn
misunderstanding: correct=nn, new=vbg
visited: correct=vbn, new=vbd
enough: correct=ap, new=qlp
out: correct=in, new=rp
to: correct=in, new=to
limited: correct=vbd, new=vbn
fight: correct=vb, new=nn
besides: correct=in, new=rb
flared: correct=vbn, new=vbd
anorthic: correct=jj, new=nn
assist: correct=nn, new=vb
north: correct=nr, new=jj
that: correct=dt, new=cs
cultivated: correct=vbd, new=vbn
eccentricities: correct=nns, new=nn
so: correct=cs, new=ql
along: correct=rb, new=in
capone's: correct=np$, new=nn
sande's: correct=np$, new=nn
wales': correct=np$, 

future: correct=jj, new=nn
off: correct=in, new=rp
toppled: correct=vbn, new=vbd
it: correct=ppo, new=pps
binding: correct=nn, new=vbg
particularly: correct=ql, new=rb
caused: correct=vbd, new=vbn
before: correct=rb, new=cs
to: correct=in, new=to
that: correct=wps, new=cs
world-shattering: correct=jj, new=nn
how: correct=wql, new=wrb
her: correct=ppo, new=pp$
short-cut: correct=jj, new=nn
brown: correct=jj, new=np
tilted: correct=vbd, new=vbn
far: correct=ql, new=rb
considered: correct=vbd, new=vbn
entered: correct=vbn, new=vbd
well: correct=uh, new=rb
as: correct=ql, new=cs
that: correct=dt, new=cs
that: correct=dt, new=cs
noticed: correct=vbn, new=vbd
right: correct=ql, new=jj
police: correct=nn, new=nns
ring: correct=vb, new=nn
as: correct=ql, new=cs
so: correct=rb, new=ql
sonorities: correct=nns, new=nn
to: correct=in, new=to
needed: correct=vbd, new=vbn
one: correct=pn, new=cd
to: correct=in, new=to
understanding: correct=vbg, new=nn
mutually: correct=ql, new=rb
covers: correct=vb

...

it: correct=ppo, new=pps
it: correct=ppo, new=pps
fall: correct=vb, new=nn
dangle: correct=vb, new=nn
her: correct=ppo, new=pp$
bottom: correct=jj, new=nn
above: correct=rb, new=in
identified: correct=vbd, new=vbn
such: correct=abl, new=jj
franco-german: correct=jj, new=nn
just: correct=ql, new=rb
to: correct=in, new=to
north: correct=nr, new=jj
much: correct=ql, new=ap
once: correct=cs, new=rb
skilfully: correct=rb, new=nn
to: correct=in, new=to
may: correct=np, new=md
change: correct=vb, new=nn
touched: correct=vbn, new=vbd
for: correct=cs, new=in
firm: correct=jj, new=nn
had: correct=hvn, new=hvd
that: correct=dt, new=cs
along: correct=rb, new=in
say: correct=uh, new=vb
after: correct=cs, new=in
it: correct=ppo, new=pps
over: correct=rp, new=in
over: correct=rp, new=in
position: correct=vb, new=nn
gains: correct=vbz, new=nns
parrots: correct=nns, new=nn
struck: correct=vbn, new=vbd
ain't: correct=bem, new=bez
lay: correct=vb, new=vbd
walk: correct=nn, new=vb
so: correct=cs, new=ql
s

trace: correct=vb, new=nn
minute: correct=jj, new=nn
to: correct=in, new=to
stands: correct=nns, new=vbz
attempted: correct=vbd, new=vbn
well: correct=uh, new=rb
that: correct=dt, new=cs
54,320: correct=cd, new=nn
642: correct=cd, new=nn
sojourner: correct=np, new=nn
made: correct=vbd, new=vbn
break: correct=nn, new=vb
denver's: correct=np$, new=nn
jarred: correct=vbn, new=vbd
grayson: correct=np, new=nn
it: correct=ppo, new=pps
right: correct=rb, new=jj
it: correct=ppo, new=pps
passing: correct=nn, new=vbg
act: correct=vb, new=nn
that: correct=wps, new=cs
plague: correct=vb, new=nn
signed: correct=vbd, new=vbn
off: correct=in, new=rp
almost: correct=ql, new=rb
however: correct=wrb, new=rb
to: correct=in, new=to
rationalized: correct=vbn, new=nn
sophomores: correct=nns, new=nn
exhibit: correct=nn, new=vb
standard: correct=nn, new=jj
for: correct=cs, new=in
crystallize: correct=vb, new=nn
god: correct=nn, new=np
only: correct=ap, new=rb
that: correct=dt, new=cs
loeser: correct=np, new=n

lettered: correct=vbd, new=nn
slung: correct=vbd, new=vbn
out: correct=in, new=rp
curdling: correct=vbg, new=nn
to: correct=in, new=to
however: correct=wrb, new=rb
caused: correct=vbd, new=vbn
effect: correct=vb, new=nn
it: correct=ppo, new=pps
it: correct=ppo, new=pps
pretty: correct=jj, new=ql
sunday-school: correct=jj, new=nn
breakups: correct=nns, new=nn
to: correct=in, new=to
de: correct=np, new=in
most: correct=ap, new=ql
akita: correct=np, new=nn
prefectures: correct=nns, new=nn
much: correct=ql, new=ap
japanese: correct=np, new=jj
coloring: correct=nn, new=vbg
warmed: correct=vbn, new=vbd
honey-in-the-sun: correct=jj, new=nn
so: correct=rb, new=ql
occupied: correct=vbd, new=vbn
after: correct=cs, new=in
made: correct=vbd, new=vbn
suffering: correct=nn, new=vbg
it: correct=ppo, new=pps
to: correct=in, new=to
occurred: correct=vbn, new=vbd
suffered: correct=vbn, new=vbd
made: correct=vbd, new=vbn
suffering: correct=nn, new=vbg
occurred: correct=vbn, new=vbd
there: correct=rb, new

daily: correct=rb, new=jj
until: correct=in, new=cs
one: correct=pn, new=cd
certain: correct=ap, new=jj
thinking: correct=nn, new=vbg
submarine: correct=jj, new=nn
hurt: correct=nn, new=vb
less: correct=ql, new=ap
to: correct=in, new=to
that: correct=dt, new=cs
minimum: correct=nn, new=jj
to: correct=in, new=to
fiercest: correct=jjt, new=nn
joey: correct=np, new=nn
slim: correct=np, new=jj
divarco: correct=np, new=nn
monk: correct=np, new=nn
allegretti: correct=np, new=nn
arianism: correct=np, new=nn
teddy: correct=np, new=nn
herrick: correct=np, new=nn
banks: correct=np, new=nns
banks: correct=np, new=nns
left: correct=vbd, new=vbn
over: correct=rp, new=in
to: correct=in, new=to
'd: correct=hvd, new=md
best: correct=rbt, new=jjt
that: correct=dt, new=cs
just: correct=ql, new=rb
as: correct=ql, new=cs
one: correct=pn, new=cd
figured: correct=vbn, new=vbd
it: correct=ppo, new=pps
you: correct=ppo, new=ppss
vote: correct=vb, new=nn
south: correct=jj, new=nr
african: correct=np, new=jj
an

along: correct=rb, new=in
comin': correct=nn, new=vbg
raid: correct=vb, new=nn
utopia: correct=nn, new=np
too: correct=rb, new=ql
down: correct=in, new=rp
uninvolved: correct=jj, new=nn
republicans': correct=nps$, new=nn
included: correct=vbd, new=vbn
to: correct=in, new=to
$214: correct=nns, new=nn
reports: correct=vbz, new=nns
st.: correct=np, new=nn
st.: correct=np, new=nn
to: correct=in, new=to
shopping: correct=nn, new=vbg
file: correct=vb, new=nn
file: correct=vb, new=nn
progress: correct=vb, new=nn
outside: correct=jj, new=in
help: correct=nn, new=vb
use: correct=vb, new=nn
it: correct=ppo, new=pps
cross: correct=vb, new=nn
bedsprings: correct=nns, new=nn
olive: correct=np, new=jj
heideman: correct=np, new=nn
until: correct=in, new=cs
there: correct=rb, new=ex
while: correct=nn, new=cs
servitors: correct=nns, new=nn
her: correct=ppo, new=pp$
to: correct=in, new=to
to: correct=in, new=to
hoisted: correct=vbd, new=vbn
to: correct=in, new=to
senator's: correct=nn$, new=nn
up: corre

The main issue with misclassifications is that it's hard for the model to recognise different types/conjugations. For example, it confuses vbn and vbd quite a lot. For that reason the following rules will be implemented:

- convert vbn to vbd during testing
- convert jjt o rb
- pps to ppo
- ppss to ppo
- do something about nn

In [27]:
train_all

{'``': [('``', 7713)],
 'he': [('pps', 8551), ('nil', 2)],
 'almost': [('rb', 242), ('ql', 138)],
 'brought': [('vbd', 120), ('vbn', 104)],
 'it': [('pps', 5421), ('ppo', 2465), ('uh', 1)],
 'back': [('rb', 651), ('nn', 155), ('jj', 25), ('vb', 19)],
 'all': [('abn', 2380), ('ql', 210)],
 'the': [('at', 60980), ('nil', 3)],
 'way': [('nn', 784), ('ql', 6), ('rb', 3)],
 "''": [("''", 7678)],
 '.': [('.', 43009)],
 'there': [('ex', 1993), ('rb', 505)],
 'is': [('bez', 8851), ('nil', 1)],
 'a': [('at', 20047), ('np', 53), ('nn', 40), ('in', 6), ('nil', 3)],
 'vast': [('jj', 55)],
 'difference': [('nn', 125)],
 'between': [('in', 629), ('rb', 1)],
 'community': [('nn', 201)],
 'of': [('in', 31805), ('nil', 2)],
 'reconciliation': [('nn', 1)],
 'which': [('wdt', 3123), ('nil', 1)],
 'new': [('jj', 1405)],
 'testament': [('nn', 29)],
 'describes': [('vbz', 19)],
 'and': [('cc', 25152), ('or', 16), ('nil', 2)],
 'congeniality': [('nn', 1)],
 'found': [('vbn', 233), ('vbd', 229)],
 'in': [('in

Based on the full training dataset the following rules can be implemented:

- label word 'to' as 'in' instead of 'to'
- for every word whose most occuring tag is 'cs', change it to the second most occuring tag
- label 'it' as 'ppo'and not 'pps'
- nn to vb because vb is one of the most occuring
- label 'you' as ppo instead of ppss

In [28]:
train_transformed = {}

for w in train_all:
    if w == 'to':
        for tuples in w:
            if tuples[0] == 'in':
                train_transformed[w] = tuples[0]
            continue
        continue
    if train_all[w][0][0] == 'cs' and  len(train_all[w]) > 1:
        train_transformed[w] = train_all[w][1][0]
        continue
    if w == 'it':
        train_transformed[w] = 'ppo'
        continue
    if w == 'you':
        train_transformed[w] = 'ppo'
        continue
    train_transformed[w] = train_all[w][0][0]

In [29]:
train_transformed

{'``': '``',
 'he': 'pps',
 'almost': 'rb',
 'brought': 'vbd',
 'it': 'ppo',
 'back': 'rb',
 'all': 'abn',
 'the': 'at',
 'way': 'nn',
 "''": "''",
 '.': '.',
 'there': 'ex',
 'is': 'bez',
 'a': 'at',
 'vast': 'jj',
 'difference': 'nn',
 'between': 'in',
 'community': 'nn',
 'of': 'in',
 'reconciliation': 'nn',
 'which': 'wdt',
 'new': 'jj',
 'testament': 'nn',
 'describes': 'vbz',
 'and': 'cc',
 'congeniality': 'nn',
 'found': 'vbn',
 'in': 'in',
 'average': 'jj',
 'church': 'nn',
 'building': 'nn',
 'scene': 'nn',
 ',': ',',
 'course': 'nn',
 'should': 'md',
 'be': 'be',
 'nine': 'cd',
 'miles': 'nns',
 'northwest': 'nr',
 'centralia': 'np',
 'illinois': 'np',
 'geographical': 'jj',
 'center': 'nn',
 'population': 'nn',
 'according': 'in',
 'census': 'nn',
 'leading': 'vbg',
 'his': 'pp$',
 'pony': 'nn',
 'hurried': 'vbd',
 'that': 'dt',
 'not': '',
 'remounting': 'vbg',
 'till': 'in',
 'was': 'bedz',
 'well': 'rb',
 'below': 'in',
 'level': 'nn',
 'surrounding': 'vbg',
 'range': 'nn

In [30]:
new_labels_transformed = []

for w in test:
    if w in train_transformed:
        new_labels_transformed.append(train_transformed[w])
    else:
        new_labels_transformed.append('nn')

In [31]:
acc = accuracy(new_labels_transformed, test_correct_tags)
print("Accuracy: %s" % acc)

Accuracy: 88.53101693663697


# Question 2c

In [32]:
def calc_transitions(tags):
    transitions = defaultdict(Counter)
    bgram = nltk.ngrams(tags, 2)
    for tag1, tag2 in bgram:
        transitions[tag1][tag2] += 1
    for from_tag in transitions:
        count = 0
        for to_tag in transitions[from_tag]:
            count += transitions[from_tag][to_tag]
        for to_tag in transitions[from_tag]:
            transitions[from_tag][to_tag] = transitions[from_tag][to_tag] / count
    return transitions

In [33]:
train_trans = calc_transitions(train_tags)

In [34]:
min_weight = 0.01
def viterbi(sentence_tokens, unique_tag, start_p, trans_p, emit_p):
    vit = [{}]
    word = sentence_tokens[0]
    for token in unique_tag:
        val = max(emit_p[token][word], min_weight)
        vit[0][token] = [ start_p[token] * val, None ]
        
    for t in range(1, len(sentence_tokens)):
        vit.append({})
        prev_state = vit[t-1]
        word = sentence_tokens[t]
        for token in unique_tag:
            prev_st_selected = unique_tag[0]
            max_tr_prob = float("-inf")
            for prev_st in unique_tag:
                tr_prob = prev_state[prev_st][0] * trans_p[prev_st][token]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
            val = max(emit_p[token][word], min_weight)
            max_prob = max_tr_prob * val
            vit[t][token] = [max_prob, prev_st_selected]
    opt = []
    max_prob = max(value[0] for value in vit[-1].values())
    previous = None
    for st, data in vit[-1].items():
        if data[0] == max_prob:
            opt.append(st)
            previous = st
            break
    for t in range(len(vit) - 2, -1, -1):
        previous = vit[t + 1][previous][1]
        opt.insert(0, previous)
    return opt

In [35]:
sentence = train_tokens[1:9]

predicted_tags = viterbi(sentence, train_unique_tags, train_tag_count_norm, train_trans, train_tag_token_count)

print("Sentence: ", sentence)
print("Predicted:", predicted_tags)
print("Acctual:  ", train_tags[1:9])

Sentence:  ['he', 'almost', 'brought', 'it', 'back', 'all', 'the', 'way']
Predicted: ['pps', 'md', 'vb', 'ppo', 'in', 'abn', 'at', 'nn']
Acctual:   ['pps', 'rb', 'vbd', 'ppo', 'rb', 'abn', 'at', 'nn']


In [36]:
sentence = ['i', 'have', 'a', 'dog']
predicted_tags = viterbi(sentence, train_unique_tags, train_tag_count_norm, train_trans, train_tag_token_count)

print("Sentence: ", sentence)
print("Predicted:", predicted_tags)

Sentence:  ['i', 'have', 'a', 'dog']
Predicted: ['ppss', 'hv', 'at', 'nn']


In [37]:
test_predicted_tags = []
for sentences in test_sentences:
    if len(sentences) > 0:
        sen_predicted = viterbi(sentences, train_unique_tags, train_tag_count_norm, train_trans, train_tag_token_count)
        test_predicted_tags.extend(sen_predicted)
    
acc = accuracy(test_predicted_tags, test_correct_tags)
print("Accuracy: %s" % acc)

Accuracy: 59.70315573247766


In [38]:
train_tag_token_count

defaultdict(collections.Counter,
            {'``': Counter({'``': 1.0}),
             'pps': Counter({'he': 0.5164582955849489,
                      'she': 0.15594612550582834,
                      'it': 0.32741438666425077,
                      'thee': 6.03974149906384e-05,
                      '': 6.03974149906384e-05,
                      'hytt': 6.03974149906384e-05}),
             'rb': Counter({'almost': 0.007555888597477208,
                      'back': 0.020325964780816785,
                      'well': 0.018296490570750595,
                      'now': 0.03556263269639066,
                      'then': 0.03662420382165605,
                      'hereunto': 0.00015611340077432246,
                      'suddenly': 0.004121393780442113,
                      'away': 0.012270513300861745,
                      'around': 0.006400649431747221,
                      'quickly': 0.0024978144123891594,
                      'never': 0.018920944173847883,
                      'a

In [39]:
sent = test_sentences[0]
corr = test_correct_tags[0:len(sent)]
sen_predicted = viterbi(sent, train_unique_tags, train_tag_count_norm, train_trans, train_tag_token_count)
print(sent)
print(corr)
print(sen_predicted)
print(test_tokens[0:len(sent)])

['in', 'great', 'expectations', 'the', 'hands', 'become', 'almost', 'an', 'obsession', '.']
['in', 'jj', 'nns', 'at', 'nns', 'vb', 'rb', 'at', 'nn', '.']
['in', 'nn', 'in', 'at', 'jj', 'nn', 'in', 'at', 'nn', '.']
['in', 'great', 'expectations', 'the', 'hands', 'become', 'almost', 'an', 'obsession', '.']


# Question 2d

In [43]:
def beam_decode(seq_map, n):
    ordered = sorted(seq_map.items(), key=lambda x: x[1])
    last_n = ordered[len(ordered) - n:]
    return [t[0] for t in last_n] 

def viterbi_beam(sentence_tokens, beam_weight, unique_tag, start_p, trans_p, emit_p):
    vit = [{}]
    word = sentence_tokens[0]
    for token in unique_tag:
        val = max(emit_p[token][word], min_weight)
        vit[0][token] = [ start_p[token] * val, None ]
        
    for t in range(1, len(sentence_tokens)):
        vit.append({})
        prev_state = vit[t-1]
        word = sentence_tokens[t]
        promising_tags = beam_decode(prev_state, beam_weight)
        for token in unique_tag:
            prev_st_selected = unique_tag[0]
            max_tr_prob = float("-inf")
            for prev_st in promising_tags:
                tr_prob = prev_state[prev_st][0] * trans_p[prev_st][token]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
            val = max(emit_p[token][word], min_weight)
            max_prob = max_tr_prob * val
            vit[t][token] = [max_prob, prev_st_selected]
    opt = []
    max_prob = max(value[0] for value in vit[-1].values())
    previous = None
    for st, data in vit[-1].items():
        if data[0] == max_prob:
            opt.append(st)
            previous = st
            break
    for t in range(len(vit) - 2, -1, -1):
        previous = vit[t + 1][previous][1]
        opt.insert(0, previous)
    return opt

In [44]:
sentence = train_tokens[1:9]

predicted_tags = viterbi_beam(sentence, 10, train_unique_tags, train_tag_count_norm, train_trans, train_tag_token_count)

print("Sentence: ", sentence)
print("Predicted:", predicted_tags)
print("Acctual:  ", train_tags[1:9])

Sentence:  ['he', 'almost', 'brought', 'it', 'back', 'all', 'the', 'way']
Predicted: ['pps', 'md', 'vb', 'ppo', 'in', 'abn', 'at', 'nn']
Acctual:   ['pps', 'rb', 'vbd', 'ppo', 'rb', 'abn', 'at', 'nn']


In [46]:
test_predicted_tags = []
for sentences in test_sentences:
    if len(sentences) > 0:
        sen_predicted = viterbi_beam(sentences, 10, train_unique_tags, train_tag_count_norm, train_trans, train_tag_token_count)
        test_predicted_tags.extend(sen_predicted)
    
acc = accuracy(test_predicted_tags, test_correct_tags)
print("Accuracy: %s" % acc)

Accuracy: 61.690844280948134


In [ ]:
#Accuracy: 61.690844280948134